In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")


In [ ]:
embeddings = load_embeddings_file("../../datasets/embeddings/GoogleNews-vectors-negative300.bin.gz", "word2vec", True)



In [3]:
description = "Sets the pivot point for scale operations."
method = "void setPivotX(float pivotX)"
class_ = "android.support.constraint.helper.Layer"

In [15]:
def extract_method_name(text):
    method_name = text.split("(")[0].split()[1]
    parts = []
    substr = ""
    for c in method_name:
        if c.isupper():
            if substr:
                parts.append(substr.lower())            
            substr = c
        else:
            substr += c
    parts.append(substr.lower())    
    return parts

def extract_class_name(text):
    class_name = text.split(".")[-1]
    parts = []
    substr = ""
    for c in class_name:
        if c.isupper():
            if substr:
                parts.append(substr.lower())
            substr = c
        else:
            substr += c
    parts.append(substr.lower())    
    return parts

def extract_dobj(text):
    doc = nlp(text)
    for token in doc:
        if token.dep_ == "dobj":
            return token.text.lower()
        



In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
def remove_stopwords(word_list):
    filtered_sentence = [w for w in word_list if not w in stop_words] 
    return filtered_sentence

In [17]:
raw_method_name = extract_method_name(method)
filtered_method_name = remove_stopwords(raw_method_name[1:])
method_name = " ".join(filtered_method_name)

In [18]:
raw_class_name = extract_class_name(class_)
class_name = " ".join(raw_class_name)

In [9]:
class_name

'layer'

In [19]:
method_name

'pivot x'

In [20]:
filtered_method_name

['pivot', 'x']

In [21]:
extract_dobj(description)

'point'

In [22]:
import os
import numpy as np

from gensim.models import KeyedVectors
from nltk import word_tokenize


In [23]:
def load_embeddings_file(file_name, embedding_type, lower):
    """TODO"""
    if not os.path.isfile(file_name):
        raise Exception("{} does not exist".format(file_name))
    words = None
    if embedding_type == "word2vec":
        model = KeyedVectors.load_word2vec_format(
            file_name, binary=True, unicode_errors="ignore"
        )
        words = model.index2entity
    else:
        raise Exception("Unknown Embedding Type")

    if lower:
        vectors = {word.lower(): model[word] for word in words}
    else:
        vectors = {word: model[word] for word in words}
    return vectors


In [24]:
from nltk.tokenize import word_tokenize

def sentence_embeddings(sentence, embeddings):
    s_vectors = [embeddings[word] for word in word_tokenize(sentence) if word in embeddings ]
    embedding = np.sum(s_vectors, axis=0)
    return embedding

In [25]:
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
def algorithm(description, method, class_, embeddings):
    obj = extract_dobj(description)
    
    raw_method_name = extract_method_name(method)
    filtered_method_name = remove_stopwords(raw_method_name[1:])
    method_name = " ".join(filtered_method_name)
    
    raw_class_name = extract_class_name(class_)
    class_name = " ".join(raw_class_name)
    
    name = ""
    if method_name:
        name = method_name
    else:
        name = class_name
    
    if name:
        obj_embedding = sentence_embeddings(obj, embeddings)
        method_embedding = sentence_embeddings(name, embeddings)
        
        similarity = dot(obj_embedding.tolist(), method_embedding.tolist())/(norm(obj_embedding.tolist())*norm(method_embedding.tolist()))
        print("{}:{}:{}".format(obj, name, similarity))
    
    
        

In [26]:
algorithm(description, method, class_, embeddings)

NameError: name 'embeddings' is not defined

In [1]:
import spacy
from benepar.spacy_plugin import BeneparComponent


ImportError: Traceback (most recent call last):
  File "/home/huseyinalecakir/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/huseyinalecakir/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/huseyinalecakir/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/huseyinalecakir/anaconda3/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/huseyinalecakir/anaconda3/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: /lib64/libm.so.6: version `GLIBC_2.23' not found (required by /home/huse

In [30]:
nlp = spacy.load("en_core_web_lg")


In [32]:
nlp.add_pipe(BeneparComponent('benepar_en'))


AttributeError: module 'tensorflow' has no attribute 'GraphDef'

In [6]:
doc = nlp("The time for action is now. It's never too late to do something.")


In [7]:
doc.sents

In [11]:
sents = list(doc.sents)

In [12]:
sent = sents[0]

In [13]:
print(sent._.parse_string)

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


In [14]:
print(sent._.labels)

('S',)


In [20]:
list(sent._.children)[0]

The time for action

In [ ]:
print(list(sent._.children)[0])

In [24]:
import benepar
parser = benepar.Parser("benepar_en2")
tree = parser.parse("Short cuts make long delays.")

In [25]:
tree.pretty_print()

                S                      
        ________|____________________   
       |             VP              | 
       |         ____|____           |  
       NP       |         NP         | 
   ____|___     |     ____|____      |  
  JJ      NNS  VBP   JJ       NNS    . 
  |        |    |    |         |     |  
Short     cuts make long     delays  . 



In [38]:
def extract_vp(parent):
    for node in parent:
        if type(node) is nltk.Tree:
            label = node.label()
            if label == "VP":
                surface = " ".join(node.leaves())
                return surface

In [39]:
extract_vp(tree)

'make long delays'